## GENERATING A SONG

In [1]:
import os
import music21
from music21 import converter, note, chord
import pickle
import numpy as np
from keras.models import load_model

2024-01-05 17:03:26.902400: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-01-05 17:03:27.078731: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-01-05 17:03:27.078788: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-01-05 17:03:27.106521: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-01-05 17:03:27.167112: I tensorflow/core/platform/cpu_feature_guar

In [2]:
model = load_model('model/')

In [3]:
# Define constants
SEQUENCE_LENGTH = 100  # Adjust as needed
EPOCHS = 100  # Adjust as needed
BATCH_SIZE = 128  # Adjust as needed
OUTPUT_FILE = 'output.mid'

# Load Chopin MIDI files
data_folder = 'data'
files = [os.path.join(data_folder, file) for file 
         in os.listdir(data_folder) if file.endswith('.midi') or file.endswith('.mid')]

def get_notes():
    """ Get all the notes and chords from the MIDI files in the specified directory """
    notes = []

    for file in files:
        midi = converter.parse(file)
        print("Parsing %s" % file)
        notes_to_parse = midi.flatten()
        for element in notes_to_parse:
            if isinstance(element, note.Note):
                notes.append(str(element.pitch))
            elif isinstance(element, chord.Chord):
                notes.append('.'.join(str(n) for n in element.pitches))
    return notes

notes_file_path = 'data/notes'
notes = []

if os.path.exists(notes_file_path):
    # Loading notes from the file
    with open('data/notes', 'rb') as filepath:
        notes = pickle.load(filepath)
else:
    # createing a list of ntoes
    notes = get_notes()
    # saving notes to the file:
    with open('data/notes', 'wb') as filepath:
        pickle.dump(notes, filepath)

# Create input sequences and corresponding output
unique_notes = sorted(set(notes))
note_to_int = dict((note, number) for number, note in enumerate(unique_notes))
int_to_note = dict((number, note) for number, note in enumerate(unique_notes))

input_sequences = []
output_sequences = []
for i in range(len(notes) - SEQUENCE_LENGTH):
    sequence_in = notes[i:i + SEQUENCE_LENGTH]
    sequence_out = notes[i + SEQUENCE_LENGTH]
    input_sequences.append([note_to_int[char] for char in sequence_in])
    output_sequences.append(note_to_int[sequence_out])


In [115]:
start = np.random.randint(0, len(input_sequences) - 1)

In [118]:
pattern = input_sequences[start]
prediction_output = []
number_of_notes_to_generate = 100
for i in range(number_of_notes_to_generate):
    input_sequence = np.reshape(pattern, (1, len(pattern), 1))
    input_sequence = input_sequence / float(len(unique_notes))

    prediction = model.predict(input_sequence, verbose=0)
    index = np.argmax(prediction)
    result = int_to_note[index]
    print(f"{i}/{number_of_notes_to_generate} | {result}")
    prediction_output.append(result)
    pattern.append(index)
    pattern = pattern[1:]

0/100 | F#3
1/100 | B-4
2/100 | G#4
3/100 | C#3
4/100 | F#4
5/100 | F#2
6/100 | F#4
7/100 | C#3
8/100 | E-5
9/100 | F#3
10/100 | E4.C#5
11/100 | B2
12/100 | F#4
13/100 | F#3
14/100 | B-4
15/100 | E-4
16/100 | F#4
17/100 | B3
18/100 | G4
19/100 | C#3
20/100 | F#4
21/100 | F#2
22/100 | F#4
23/100 | C#3
24/100 | E-5
25/100 | F#3
26/100 | E4.C#5
27/100 | B2
28/100 | E-5.F#5
29/100 | F#3
30/100 | E4.C#5
31/100 | B2
32/100 | F#4
33/100 | F#3
34/100 | B4.E-5
35/100 | E-3
36/100 | F#4
37/100 | B3
38/100 | G4
39/100 | C#3
40/100 | F#4
41/100 | F#2
42/100 | F#4
43/100 | C#3
44/100 | E-5
45/100 | F#3
46/100 | E4.C#5
47/100 | B2
48/100 | F#4
49/100 | F#3
50/100 | B-4
51/100 | B2
52/100 | F#4
53/100 | F#3
54/100 | B-4
55/100 | B3
56/100 | F#4
57/100 | E-3
58/100 | B-4
59/100 | E-4
60/100 | F#4
61/100 | E-4
62/100 | B4
63/100 | E-4
64/100 | F#4
65/100 | E3
66/100 | B-3
67/100 | C#4
68/100 | F#4
69/100 | B3
70/100 | B-4
71/100 | E-4
72/100 | F#4
73/100 | F#3
74/100 | B-4
75/100 | E-4
76/100 | F#4
77/

In [120]:
# Convert output to MIDI file
output_midi = music21.stream.Stream()
shift = 0
for pattern in prediction_output:
    if '.' in pattern:   # it's a chord
        notes_in_chord = pattern.split('.') # notes for the chord (numbers)
        chord_notes = [music21.note.Note() for _ in notes_in_chord] # real notes
        for i, note_in_chord in enumerate(notes_in_chord):
            chord_notes[i].pitch = music21.pitch.Pitch(note_in_chord)
        chord = music21.chord.Chord(chord_notes, quarterlength=0.5) # the chord per se
        output_midi.insert(shift, chord) # adding the chord to the stream
        shift += 0.3
    else:   # it's a note
        note = music21.note.Note(quarterlength=0.5)
        note.pitch = music21.pitch.Pitch(pattern)
        output_midi.insert(shift, note) # adding the note to the stream
        shift += 0.3

output_midi.write('midi', fp=OUTPUT_FILE)

'output.mid'